In [3]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table_experiments as dt
import plotly.graph_objs as go
import pandas as pd
import flask
from geopy.geocoders import Nominatim 
import itertools
from itertools import *
import os
import numpy as np
from dash.dependencies import Input, Output, State, Event
#import json


In [8]:

# Reading the dataset in '.xlsx' file 
def import_excel(
    xlxs_file_nme, sheet_num ,col_nme = '' , sheet_idx=False, file_loc=True):
    """
    Import and convert .xlxs file to a dataframe 
    Inputs:
    ________________
     - xlxs_file_nme: str
         .xlxs file name
     - sheet_num: int / str
         name (or number) of the sheet inside .xlxs
     - col_nme: str
         when sheet_idx is "True", add column name to set dataframe index 
     - sheet_idx: bol
         set dataframe index according to any specified column within the datframe , default "False" 
     - file_loc:
    Outputs:
    ________________
     - dataframe of dataset
    """
    if file_loc:
        file_path = os.path.abspath(xlxs_file_nme)
        xlsx = pd.ExcelFile(file_path)
        sheet1 = xlsx.parse(sheet_num)
        if sheet_idx:
            sheet1.index = sheet1[col_nme]
            new_df_beach_0 = sheet1.iloc[:]
            new_df_beach_0.fillna(value= 'NaN', inplace=True)
            return new_df_beach_0
        else:
            new_df_beach_0 = sheet1.iloc[:]
            new_df_beach_0.reset_index(inplace= True)
            new_df_beach_0.drop(['index'], axis=1, inplace = True)
            new_df_beach_0.fillna(value= 'NaN', inplace=True)
            return new_df_beach_0
    else:
        xlsx = pd.ExcelFile(xlxs_file_nme)
        sheet1 = xlsx.parse(sheet_num)
        if sheet_idx:
            sheet1.index =  sheet1[col_nme]
            new_df_beach_0 = sheet1.iloc[:]
            new_df_beach_0.fillna(value= 'NaN', inplace=True)
            return new_df_beach_0
        else:
            new_df_beach_0 = sheet1.iloc[:]
            new_df_beach_0.reset_index(inplace= True)
            new_df_beach_0.drop(['index'], axis=1, inplace = True)
            new_df_beach_0.fillna(value= 'NaN', inplace=True)
            return new_df_beach_0

## Add different info to the map marker
def df_map(
    df, location_col, name_col, beach, proj, client, status, 
    deadline, code):
    def poptext_3(
        df, location_col, name_col, beach, proj, client, status, 
        deadline, code, x):
    
        new_df = df[
            [location_col, name_col, beach, proj, client, status, 
             deadline, code]
        ].groupby([location_col, name_col, beach, proj, client, status, 
                   deadline, code]).count()
        info_lst = []
        for info in new_df.index:
            if info[0] == x:
                yield 'Name: {0} , Beach: {1} , Project: {2}, Client: {3},\
                Status: {4}, Deadline: {5} , Case code: {6}'.format(
                    info[1], info[2], info[3], info[4], info[5], 
                    info[6].date(), info[7])
                
    geolocator = Nominatim()        
    countries = pd.DataFrame({'country': list(set(df[location_col]))})
    countries['lat'] = countries['country'].apply(
    lambda x: geolocator.geocode(x, timeout=15).latitude)
    countries['lon'] = countries['country'].apply(
    lambda x: geolocator.geocode(x, timeout=15).longitude)
    countries['Avrg_Days_till_Deadline'] = df.groupby(['Location'])[
        'Days until DL'].mean().round(decimals=0).values
    countries['info'] = countries['country'].apply(lambda i: list(poptext_3(
    df,location_col, name_col, beach, proj, client, status, 
        deadline, code, i)))
    return countries

In [11]:
# Builidng Dash/plotly app ´
new_df_beach = import_excel(
    'Copy of Beach Work HTMLversion.xlsx', sheet_num=0, sheet_idx = False, 
) #, sheet_idx='Name'

converted_df = df_map(new_df_beach,'Location', 'Name','Beach','Project' , 
                     'Client', 'Status' , 'Deadline', 'Case code')

converted_df

,country,lat,lon,Avrg_Days_till_Deadline,info
0,Italy,42.638426,12.674297,162.0,"[Name: Axel , Beach: yes , Project: NaN, Clie..."
1,Denmark,55.670249,10.333328,137.0,"[Name: Erik H , Beach: yes , Project: NaN, Cl..."
2,United States,39.783730,-100.445882,142.0,"[Name: Emelie , Beach: yes , Project: NaN, Cl..."
3,Sweden,59.674971,14.520858,142.0,"[Name: Alexander , Beach: yes , Project: NaN,..."
4,Finland,63.246778,25.920916,142.0,"[Name: Erik R , Beach: no , Project: NaN, Clie..."
5,United Kingdom,54.702354,-3.276575,162.0,"[Name: Anders , Beach: yes , Project: NaN, Cl..."
6,Spain,39.326234,-4.838065,142.0,"[Name: Daniel , Beach: no , Project: NaN, Clie..."
7,Germany,51.083420,10.423447,142.0,"[Name: Andreas , Beach: yes , Project: NaN, C..."
8,Russian Federation,64.686314,97.745306,142.0,"[Name: Emil , Beach: no , Project: NaN, Client..."
9,France,46.603354,1.888334,162.0,"[Name: Artin , Beach: no , Project: NaN, Clien..."


In [ ]:
# 1- Building and deploying the Dash App

app = dash.Dash()

# Bootstrap grid system for front-end
app.css.append_css({'external_url': 'https://cdn.rawgit.com/plotly/dash-app-stylesheets/2d266c578d2a6e8850ebce48fdb52759b2aef506/stylesheet-oil-and-gas.css'})

mapbox_access_token = 'pk.eyJ1Ijoib21hcmhhemltIiwiYSI6ImNqY2s5cHk3MzNyZDEycm1tanV6c3pzdGUifQ.hLOK6z98WohsI19MmNBiHw'


app.layout = html.Div([
    html.H1('Team Location'),
    html.Div(id='text-content'),
    dcc.Graph(id='map', figure={
        'data': [{
            'lat': converted_df['lat'],
            'lon': converted_df['lon'],
            'marker': {
                'size': 8,
                'opacity': 0.6,
                'color': 'red'
            },
            'customdata': converted_df['country'],
            'type': 'scattermapbox'
        }],
        'layout': {
            'mapbox': {
                'accesstoken': mapbox_access_token,
            },
            'hovermode': 'closest',
            'margin': {'l': 20, 'r': 20, 'b': 20, 't': 60},
            'autosize': True
        }
    })
])


@app.callback(
    dash.dependencies.Output('text-content', 'children'),
    [dash.dependencies.Input('map', 'hoverData')])
def update_text(hoverData):
    s = converted_df[converted_df['country'] == hoverData['points'][0]['customdata']]
    return html.H3(
        '{}, {} {}'.format(
            s.iloc[0]['country'],
            s.iloc[0]['info'],
            s.iloc[0]['Avrg_Days_till_Deadline']
        )
    )

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

if __name__ == '__main__':
    app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Oct/2018 16:49:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2018 16:49:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Oct/2018 16:49:44] "GET /_dash-dependencies HTTP/1.1" 200 -
[2018-10-20 16:49:44,482] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/omar/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/omar/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/omar/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/omar/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/Users/omar/anaconda3/lib/python3.6/site-packages/flask/

In [ ]:

app = dash.Dash()
mapbox_access_token = 'pk.eyJ1Ijoib21hcmhhemltIiwiYSI6ImNqY2s5cHk3MzNyZDEycm1tanV6c3pzdGUifQ.hLOK6z98WohsI19MmNBiHw'


app.layout = html.Div([
    html.Div(
        html.Pre(id='lasso', style={'overflowY': 'scroll', 'height': '100vh'}),
        className="three columns"
    ),

    html.Div(
        className="nine columns",
        children=dcc.Graph(
            id='graph',
            figure={
                'data': [{
                    'lat': converted_df['lat'], 'lon': converted_df['lon'], 'type': 'scattermapbox',
                    'text': converted_df['info']
                }],
                'layout': {
                    'mapbox': {
                        'accesstoken': (
                            mapbox_access_token
                        )
                    },
                    'margin': {
                        'l': 20, 'r': 20, 'b': 20, 't': 60
                    },
                }
            }
        )
    )
], className="row")


app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})


@app.callback(
    Output('lasso', 'children'),
    [Input('graph', 'selectedData')])
def display_data(selectedData):
    return json.dumps(selectedData, indent=2)


if __name__ == '__main__':
    app.run_server(debug=False)

In [13]:
new_df_beach = import_excel(
    'Copy of Beach Work HTMLversion.xlsx', sheet_num=0, sheet_idx = False)
geolocator = Nominatim()
new_df_beach['Lat'] = list(itertools.chain(new_df_beach['Location'].apply(lambda x: geolocator.geocode(x, timeout=15).latitude)))
new_df_beach['Lon'] = list(itertools.chain(new_df_beach['Location'].apply(lambda x: [geolocator.geocode(x, timeout=15).longitude])))
new_df_beach['idx'] = list(i+1 for i in new_df_beach.index)
cities = list(itertools.chain(new_df_beach['Location'].values))
info_1 = new_df_beach['Location'].apply(lambda i: poptext_2(new_df_beach, 'Location', 'Name','Deadline', i))
lat = list(itertools.chain(set(new_df_beach['Location'].apply(lambda x: geolocator.geocode(x, timeout=15).latitude))))
lon = list(itertools.chain(set(new_df_beach['Location'].apply(lambda x: geolocator.geocode(x, timeout=15).longitude))))


In [ ]:

app = dash.Dash()

app.layout = html.Div([
    html.H1('STHLM Office Team Map'),
    html.Div(id='text-content'),
    dcc.Graph(id='map', figure={
        'data': [{
            'lat': lat,
            'lon': lon,
            'marker': {
                #'color': new_df_beach['Deadline'],
                'size': 8,
                'opacity': 0.6
            },
            'customdata': list(set(cities)),
            'type': 'scattermapbox'
        }],
        'layout': {
            'mapbox': {
                'accesstoken': 'pk.eyJ1Ijoib21hcmhhemltIiwiYSI6ImNqY2s5cHk3MzNyZDEycm1tanV6c3pzdGUifQ.hLOK6z98WohsI19MmNBiHw',
                'center': {'lat': np.mean(lat), 'lon': np.mean(lon)}
            },
            'hovermode': 'closest',
            'margin': {'l': 20, 'r': 20, 'b': 20, 't': 60}
        }
    })
])


app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

if __name__ == '__main__':
    app.run_server(debug=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jan/2018 22:04:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2018 22:04:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2018 22:04:49] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2018 22:04:50] "GET /favicon.ico HTTP/1.1" 200 -


0    ['Russian Federation'].['Emil', 'Olle', 'Sean B']
1           ['France'].['Artin', 'Justin', 'Rikard S']
2         ['Germany'].['Andreas', 'Jonas', 'Rikard L']
3                      ['Finland'].['Erik R', 'Oscar']
4              ['Denmark'].['Erik H', 'Omar', 'Sofia']
5       ['United Kingdom'].['Anders', 'Johan', 'Paul']
6         ['Spain'].['Daniel', 'Kristoffer', 'Sara H']
7      ['United States'].['Emelie', 'Mikko', 'Sara J']
8         ['Sweden'].['Alexander', 'Jennifer', 'Otto']
9              ['Italy'].['Axel', 'Kristin', 'Samuel']
Name: info, dtype: object

In [7]:
# {
#'col':[]
#'data': [[1], [2]]
#}


In [3]:
def new_info_model(df, location_col, name_col, beach):
    """
    Building info model to plug into map markers
    """
    def poptext_marker(df, location_col, name_col, beach, x):
            new_df_0 = df[
                [location_col, name_col, beach]
            ].groupby([location_col, name_col, beach]).count()
            return new_df_0.loc[x]
    geolocator = Nominatim() 
    countries = pd.DataFrame({
        'country': list(set(df[location_col]))})
    countries['lat'] = countries['country'].apply(
    lambda x: geolocator.geocode(x, timeout=15).latitude)
    countries['lon'] = countries['country'].apply(
    lambda x: geolocator.geocode(x, timeout=15).longitude)
    countries['Avrg_Days_till_Deadline'] = df.groupby([location_col])[
        'Days until DL'].mean().round(decimals=0).values
    countries['info'] = countries['country'].apply(
        lambda i: poptext_marker(
            df, location_col, name_col, beach, i))
    countries.index = countries['country']
    return countries
    


In [4]:
new_df = import_excel(
    'Copy of Beach Work HTMLversion.xlsx', sheet_num=0, sheet_idx = False, 
) #, sheet_idx='Name'

converted_df = new_info_model(new_df, 'Location', 'Name', 'Beach')



In [5]:
converted_df

,country,lat,lon,Avrg_Days_till_Deadline,info
country,,,,,
Germany,Germany,51.083420,10.423447,162.0,"Empty DataFrame Columns: [] Index: [(Andreas, ..."
Sweden,Sweden,59.674971,14.520858,137.0,Empty DataFrame Columns: [] Index: [(Alexander...
Finland,Finland,63.246778,25.920916,142.0,"Empty DataFrame Columns: [] Index: [(Erik R, n..."
Russian Federation,Russian Federation,64.686314,97.745306,142.0,"Empty DataFrame Columns: [] Index: [(Emil, no)..."
Spain,Spain,40.002803,-4.003104,142.0,"Empty DataFrame Columns: [] Index: [(Daniel, n..."
United Kingdom,United Kingdom,25.223228,55.159493,162.0,"Empty DataFrame Columns: [] Index: [(Anders, y..."
France,France,46.603354,1.888334,142.0,"Empty DataFrame Columns: [] Index: [(Artin, no..."
Italy,Italy,42.638426,12.674297,142.0,"Empty DataFrame Columns: [] Index: [(Axel, yes..."
Denmark,Denmark,55.670249,10.333328,142.0,"Empty DataFrame Columns: [] Index: [(Erik H, y..."


In [ ]:
app = dash.Dash()

mapbox_access_token = 'pk.eyJ1Ijoib21hcmhhemltIiwiYSI6ImNqY2s5cHk3MzNyZDEycm1tanV6c3pzdGUifQ.hLOK6z98WohsI19MmNBiHw'


app.layout = html.Div([
    html.H1('Team Location'),
    html.Div(id='text-content'),
    dcc.Graph(id='map', figure={
        'data': [{
            'lat': converted_df['lat'],
            'lon': converted_df['lon'],
            'marker': {
                'size': 8,
                'opacity': 0.6,
                'color': 'red'
            },
            'customdata': converted_df['country'],
            'type': 'scattermapbox'
        }],
        'layout': {
            'mapbox': {
                'accesstoken': mapbox_access_token,
            },
            'hovermode': 'closest',
            'margin': {'l': 20, 'r': 20, 'b': 20, 't': 60},
            'autosize': True
        }
    })
])


@app.callback(
    dash.dependencies.Output('text-content', 'children'),
    [dash.dependencies.Input('map', 'hoverData')])
def update_text(hoverData):
    s = converted_df[converted_df['country'] == hoverData['points'][0]['customdata']]
    return html.H3(
        '{}, {} {}'.format(
            #s.iloc[0]['country'],
            #s.iloc[0]['info']
            #s.iloc[0]['Avrg_Days_till_Deadline']
        )
    )

app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

if __name__ == '__main__':
    app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Feb/2018 22:02:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2018 22:02:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Feb/2018 22:02:44] "GET /_dash-dependencies HTTP/1.1" 200 -
[2018-02-01 22:02:44,852] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/omar/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/omar/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/omar/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/omar/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/Users/omar/anaconda3/lib/python3.6/site-packages/flask/

127.0.0.1 - - [01/Feb/2018 22:02:56] "POST /_dash-update-component HTTP/1.1" 500 -
